In [ ]:
# you may need to install networkx with pip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
import networkx as nx
import colorsys


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns


### Decision Trees

In [ ]:
df = pd.read_csv('gym_members_exercise_tracking.csv')
display(df)
display(df.columns)

In [ ]:
# show distribution of calories burned
sns.histplot(df['Calories_Burned'])

In [ ]:

df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})

In [ ]:
display(df['Workout_Type'].unique())

df = pd.get_dummies(df, columns=['Workout_Type'])

In [ ]:
def labelCalories(amount):
    if 0 <= amount < 500:
        return "Small"
    elif 500 <= amount < 1200:
        return "Medium"
    else:
        return "Large"

In [ ]:
df['Calories'] = df['Calories_Burned'].apply(labelCalories)

# standerdize the data
# scaler = StandardScaler()
# scaler.fit(df.drop('Calories', axis=1))

# scaled_features = scaler.transform(df.drop('Calories', axis=1))
# scaled_features_df = pd.DataFrame(scaled_features, columns=df.columns[:-1])

# scaled_features_df.drop(['Calories_Burned'], axis=1, inplace=True)

# display(scaled_features_df)

In [ ]:
cols = df.columns.tolist()
cols.remove('Calories')

corr = df[cols].corr(min_periods=15) #  'Max_BPM',  'Experience_Level', 
sns.heatmap(corr)
# create a sorted list of columns based on absolute value correlation with calories
corr = corr['Calories_Burned'].apply(abs).sort_values(ascending=False)

corr.drop('Calories_Burned', inplace=True)

display(corr)

In [ ]:
# split data into x and y
df.drop('Calories_Burned', axis=1, inplace=True)
X = df.drop(['Calories'], axis=1)
y = df['Calories']

In [ ]:
# train desiscion tree and visulize it

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np

def test_dTree(depth, features_num, tests, print_report=False):

    total_predictions = []
    total_y_test = []

    for i in range(tests):

        dtree = DecisionTreeClassifier(max_depth=depth)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        X_train = X_train[corr[:features_num].index]
        X_test = X_test[X_train.columns]

        dtree.fit(X_train, y_train)

        predictions = dtree.predict(X_test)

        total_predictions.extend(predictions)
        total_y_test.extend(y_test)

    report = classification_report(total_y_test, total_predictions, output_dict=True, zero_division=0)

    if print_report:
        print(classification_report(total_y_test, total_predictions))

    return report


def plot_tree(dtree, features):
    fig = plt.figure(figsize=(25,20))

    _ = tree.plot_tree(dtree,
                        feature_names=features,
                        class_names=['Small', 'Medium', 'Large'],
                        filled=True)

    # fig.savefig("decision_tree.png")

    plt.show()

In [ ]:
tests = 1 # changed from 100 so its will run faster

max_depth = 17
max_features = len(X.columns)

f1_scores = np.zeros((max_depth, max_features))

for i in range(max_depth):
    for j in range(max_features):
        depth = i + 1
        feature_num = j + 1

        report = test_dTree(depth, feature_num, tests)

        print(f"Depth: {depth}, Features: {feature_num}: {report['weighted avg']['f1-score']}")

        f1_scores[i][j] = report['weighted avg']['f1-score']


plt.show()

In [ ]:
sns.heatmap(f1_scores, fmt=".1f",)

best_score = np.max(f1_scores)
best_score_index = np.where(f1_scores == best_score)



plt.gca().add_patch(plt.Rectangle((best_score_index[1][0], best_score_index[0][0]), 1, 1, fill=False, edgecolor='red', lw=3))


plt.xlabel('Top N Features by correlation')
plt.ylabel('Max Depth')
plt.title('F1 Score for Decision Tree on test data')

#save

# plt.savefig("f1_score_heatmap.png")

In [ ]:
# bias the f1 scores withe a combination of teh niumber of features and the depth then plot again

f1_scores_bias = np.zeros((max_depth, max_features))

for i in range(max_depth):
    for j in range(max_features):
        depth = i + 1
        feature_num = j + 1

        f1_scores_bias[i][j] = f1_scores[i][j] - .15*((feature_num/max_features)*(depth/max_depth))

sns.heatmap(f1_scores_bias, fmt=".1f",)

best_score_b = np.max(f1_scores_bias)
best_score_index_b = np.where(f1_scores_bias == best_score_b)



plt.gca().add_patch(plt.Rectangle((best_score_index_b[1][0], best_score_index_b[0][0]), 1, 1, fill=False, edgecolor='red', lw=3))


plt.xlabel('Top N Features sorted by correlation')
plt.ylabel('Max Depth')
plt.title('F1 Score with bias for Decision Tree on test data')

#save

# plt.savefig("f1_score_heatmap_bias.png")

In [ ]:
print(f"Best F1 Score: {best_score}, Depth: {best_score_index[0][0] + 1}, Features: {best_score_index[1][0] + 1}")
print(f"Best F1 Score with bias: {best_score_b}, Depth: {best_score_index_b[0][0] + 1}, Features: {best_score_index_b[1][0] + 1}")

print(f"unbiased f1 score: {f1_scores[best_score_index_b[0][0]][best_score_index_b[1][0]]}")

In [ ]:
# create tree with best depth and features

test_dTree(best_score_index_b[0][0] + 1, best_score_index_b[1][0] + 1, 100, True)

dtree = DecisionTreeClassifier(max_depth=best_score_index_b[0][0] + 1)

dtree.fit(X, y)

plot_tree(dtree, X.columns)

### Neural Networks

In [ ]:
df = pd.read_csv('gym_members_exercise_tracking.csv')
display(df)
display(df.columns)

In [ ]:
def labelCalories(amount):
    if 0 <= amount < 500:
        return "Small"
    elif 500 <= amount < 1200:
        return "Medium"
    else:
        return "Large"

In [ ]:
df['Calories'] = df['Calories_Burned'].apply(labelCalories)
display(df)

### Age, Fat_Percentage, Height (m), 'Weight (kg)

In [ ]:
corr = df[['Age', 'Gender', 'Weight (kg)', 'Height (m)','Avg_BPM',
       'Resting_BPM', 'Session_Duration (hours)', 'Calories_Burned',
       'Workout_Type', 'Fat_Percentage', 'Water_Intake (liters)',
       'Workout_Frequency (days/week)','BMI', 'Calories']].corr(min_periods=15) #  'Max_BPM',  'Experience_Level', 
sns.heatmap(corr)

In [ ]:
# Age, Fat_Percentage, Height (m), 'Weight (kg)
X = df.drop(['Gender','Avg_BPM','Resting_BPM', 'Session_Duration (hours)', 'Calories_Burned',
             'Workout_Type', 'Water_Intake (liters)', 'Workout_Frequency (days/week)','BMI', 'Calories'],axis=1)
y = df['Calories']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
scaler = StandardScaler()

# Fit only to the training data
scaler.fit(X_train)

# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train

In [ ]:
def show_ann(mlp):
    hidden_layers_n = len(mlp.coefs_)-1
    layers_n = hidden_layers_n + 2
    input_neurons_n = len(mlp.coefs_[0])
    hidden_neurons_n = [len(mlp.coefs_[i+1]) for i in range(hidden_layers_n)]
    output_neurons_n = len(mlp.coefs_[-1][0])

    G = nx.DiGraph()
    pos = {}

    # Create the neurons of the input layer
    for i in range(input_neurons_n):
        pos['Layer0_{}'.format(i)] = (i,layers_n-1)

    for j in range(hidden_layers_n):
        # Create the neurons of the j'th hidden layer
        prev_layer = j
        cur_layer = j+1
        if (j == 0):
            prev_size = input_neurons_n
        else:
            prev_size = hidden_neurons_n[j-1]
        for i in range(hidden_neurons_n[j]):
            pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
            for k in range(prev_size):
                w = mlp.coefs_[prev_layer][k][i]
                G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    # Create the neurons of the output layer
    prev_layer = hidden_layers_n
    cur_layer = hidden_layers_n+1
    for i in range(output_neurons_n):
        pos['Layer{}_{}'.format(cur_layer,i)] = (i,layers_n-1-cur_layer)
        for k in range(hidden_neurons_n[-1]):
            w = mlp.coefs_[prev_layer][k][i]
            G.add_edge('Layer{}_{}'.format(prev_layer,k),'Layer{}_{}'.format(cur_layer,i), weight=w)

    edges = G.edges()
    colors = [colorsys.hsv_to_rgb(0 if G[u][v]['weight'] < 0 else 0.65,
                                  1,#min(1, abs(G[u][v]['weight'])),
                                  1) for u,v in edges]
    weights = [abs(G[u][v]['weight'])*2 for u,v in edges]

    nx.draw(G, pos, node_color='y', node_size=450, width=weights, edge_color=colors)
    


In [ ]:
def makeNetwork(hiddenLayer):
    # mlp = MLPClassifier(hidden_layer_sizes=(5),max_iter=500)
    mlp = MLPClassifier(hidden_layer_sizes=hiddenLayer,max_iter=50000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_test)
    print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))
    print('This dataset has {} input nodes and {} output node(s)'.format(len(X.columns), len(y.unique())))
    print('There are {} 2D arrays of coefficients, one for each layer'.format(len(mlp.coefs_)))
    print('The layers have the following number of coefficients: {}')
    for l in range(len(mlp.coefs_)):
        m = len(mlp.coefs_[l])
        n = len(mlp.coefs_[l][0])
        print('  {}: {}x{} ({} nodes feeding into a layer of {} nodes)'.format(l, m, n, m, n))
    # Print the actual coefficients
    # print(mlp.coefs_)

    print()
    print('There are {} 1D arrays of intercepts, one for each layer'.format(len(mlp.intercepts_)))
    print('Each layer has {} intercepts, one for each node'.format([len(mlp.intercepts_[l]) for l,_ in enumerate(mlp.intercepts_)]))
    show_ann(mlp)

In [ ]:
makeNetwork((13, 13, 2, 14, 11))

In [ ]:
makeNetwork((13, 13,14, 11))

In [ ]:
makeNetwork((13, 14, 11))

In [ ]:
makeNetwork((13, 11))

In [ ]:
makeNetwork((17, 14))